In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os 
hf_token = os.getenv('HUGGING_FACE_HUB_TOKEN')

### [ TEST ] LLAMA Model 

In [ ]:
from llama3 import LlamaModel

In [ ]:
# Initialize the Llama model 
model_name = "meta-llama/Llama-3.1-8B"

try: 
    print(f'Initializing {model_name}...') 
    llama = LlamaModel(
        model_name= model_name, 
        hf_token= hf_token, 
        device= 'auto',
        quantize= True
    )
    print('\u2705 Llama Model created successfully')

except Exception as e: 
    print(f'\u274c Error when creating Llama model: {e}')

In [ ]:
# Load model 
print('Load Llama model')
llama.load_model()

In [ ]:
# Test discuss with model 
test_prompts = [
    "Hello, how are you today?",
    "What is artificial intelligence?",
    "Tell me about Vietnamese food.",
    "Explain machine learning in simple terms."
]

for prompt in test_prompts: 
    print(f'\n Prompt: {prompt}')
    try: 
        response = llama.generate_text(
            prompt= prompt, 
            max_new_tokens= 100, 
            temperature= 0.7
        )

        print(f'[RESPONSE]: {response}')
    except Exception as e: 
        print(f'\u247c Generate error: {e}')

### [ TEST ] RAG SYSTEM

In [ ]:
from rag_system import RAGSystem

In [ ]:
# Init the rag system 
menu_file_path = 'menu.json'

try: 
    rag = RAGSystem(menu_file_path)
    print(f'System status: {rag.get_stats()}')
except Exception as e: 
    print(f'Initialize failed: {e}')


In [ ]:
# Basic search 
search_queries = [
    ("phở bò", "Vietnamese beef pho"),
    ("gỏi cuốn", "Fresh spring rolls"),
    ("bún bò Huế", "Hue-style beef noodle soup"),
    ("cà phê", "Vietnamese coffee"),
    ("chả giò", "Fried spring rolls"),
    ("cơm tấm", "Broken rice"),
    ("spicy soup", "Spicy soup dishes"),
    ("grilled meat", "Grilled meat dishes"),
    ("cold drink", "Cold beverages"),
    ("appetizer", "Starter dishes")
]

for query, description in search_queries: 
    print(f'{description} : {query}')

    try: 
        results = rag.search_index(query, top_k= 3, threshold= 0.2)

        if results: 
            print(f'Found {len(results)} results')
            for i, result in enumerate(results, 1): 
                print(f"   {i}. {result['name']} (Score: {result['similarity_score']:.3f})")
                print(f"      Price: {result['price']} VND | Category: {result['category']}")
        else: 
            print('No results found')
    except Exception as e: 
        print(f'Search failed: {e}')

In [ ]:
# Test 2: Context Generation for Vietnamese Restaurant Scenarios
print("\n" + "="*50)
print("TEST 2: Context Generation for Vietnamese Restaurant Scenarios")
print("="*50)

scenarios = [
    ("Quán có những món phở gì?", "What pho dishes do you have?"),
    ("Cho tôi xem các món khai vị", "Show me appetizers"),
    ("Tôi muốn ăn món cay cay", "I want something spicy"),
    ("Có những đồ uống gì?", "What drinks are available?"),
    ("Món cơm nào có thịt nướng?", "Rice dishes with grilled meat"),
    ("Món ăn truyền thống Việt Nam", "Traditional Vietnamese food"),
    ("Đồ uống mát lạnh", "Cold refreshing drinks"),
    ("Các món có thịt heo", "Pork dishes"),
    ("Món ăn nhẹ để ăn chơi", "Light snacks/appetizers"),
    ("Bún nào ngon nhất?", "Which noodle soup is best?"),
    ("Có món chay không?", "Do you have vegetarian dishes?"),
    ("Món ăn cho trời nóng", "Food for hot weather"),
    ("Cà phê Việt Nam", "Vietnamese coffee"),
    ("Món cuốn tươi mát", "Fresh spring rolls"),
    ("Đặc sản miền Nam", "Southern Vietnamese specialties")
]

for scenario_vn, scenario_en in scenarios:
    print(f"\n💭 Vietnamese: '{scenario_vn}'")
    print(f"   English: '{scenario_en}'")
    print("-" * 40)
    
    try:
        # Test with Vietnamese query
        context = rag.get_context_for_llms(scenario_vn, top_k=3)
        print("📋 Context Generated:")
        print(context)
        
        # Check if results were found
        if "No relevant menu items found" in context:
            print("⚠️  No results with Vietnamese query, trying English...")
            context_en = rag.get_context_for_llms(scenario_en, top_k=3)
            if "No relevant menu items found" not in context_en:
                print("📋 Context with English query:")
                print(context_en)
        
    except Exception as e:
        print(f"❌ Context generation error: {e}")

### AI Waiter 

In [ ]:
from ai_chatbot import AIWaiter

In [ ]:
# Initialize the Llama model 
model_name = "meta-llama/Llama-3.1-8B"
menu_file_path = 'menu.json'

waiter = AIWaiter(menu_file_path, model_name, hf_token)

In [ ]:
test_queries = [
    "Hello! What do you recommend?",
    "Tôi muốn ăn phở. Có loại nào?",
    "I want something spicy and filling",
    "What appetizers do you have?",
    "Món nào rẻ nhất?",
    "Show me your coffee options"
]

for query in test_queries: 
    print(f'Customer: {query}')
    response = waiter.chat(query)
    print(f'Linh: {response}')
    